# Notebook 7 — Income DKL Dodged Bar Charts

Produces dodged bar charts of `dkl_bin` by income bracket for four study cities:
Chicago, Houston, Atlanta, and New York (as defined in `config.CITIES`).

**`dkl_bin`** measures how much information a particular income bin contributes
to the overall divergence between a block group and its CBSA:
$$\text{DKL}_{\text{bin}}(j) = \sum_i p_{ni|yj} \log_2\frac{p_{ni|yj}}{p_{ni}}$$

A high `dkl_bin` for a bracket means households in that bracket are unevenly
distributed across block groups relative to the CBSA average.

**Inputs**: `data/acs5_block_{year}_data/dkl_income_{yr_short}_blkgrp_all_states.parquet`  
**Years**: 2010, 2015, 2020


In [ ]:
# ── Imports ──────────────────────────────────────────────────────────────────
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

warnings.filterwarnings('ignore')

NOTEBOOK_DIR = Path().resolve()
PIPELINE_DIR = NOTEBOOK_DIR.parent
if str(PIPELINE_DIR) not in sys.path:
    sys.path.insert(0, str(PIPELINE_DIR))

from config import DATA_DIR, CITIES

sns.set_theme(style='whitegrid', font_scale=0.95)

print(f'DATA_DIR : {DATA_DIR}')
print(f'CITIES   : {CITIES}')


## 1. Load DKL Income Parquet Files

Load income DKL results for 2010, 2015, and 2020.  The files follow the naming
convention `dkl_income_{yr_short}_blkgrp_all_states.parquet`.


In [ ]:
TARGET_YEARS = [2010, 2015, 2020]


def _dkl_income_path(year: int, data_dir: Path) -> Path:
    yr_short = str(year)[2:]
    return data_dir / f'acs5_block_{year}_data' / f'dkl_income_{yr_short}_blkgrp_all_states.parquet'


def load_income_dkl(years: list, data_dir: Path) -> pd.DataFrame:
    frames = []
    for yr in years:
        path = _dkl_income_path(yr, data_dir)
        if not path.exists():
            print(f'  [WARN] Not found (skipping): {path}')
            continue
        df = pd.read_parquet(path)
        df['year'] = yr
        frames.append(df)
        print(f'  Loaded {path.name}  ({len(df):,} rows)')

    if not frames:
        raise FileNotFoundError(
            'No DKL income parquet files found.  '
            'Run pipeline/2_calculate_dkl.py and/or pipeline/3_income_2010.py first.'
        )

    dkl = pd.concat(frames, ignore_index=True)
    print(f'\nTotal rows : {len(dkl):,}')
    print(f'Columns    : {list(dkl.columns)}')
    return dkl


dkl_income = load_income_dkl(TARGET_YEARS, DATA_DIR)
dkl_income.head(3)


## 2. Filter to Study Cities

Keep only rows whose `cbsa_title` matches one of the four study cities.
We match on the first segment (before the first comma) to be robust to
minor formatting differences.


In [ ]:
# Short display names derived from config.CITIES
CITY_SHORT = {
    c: c.split(',')[0].split('-')[0].strip() for c in CITIES
}
print('City display names:')
for full, short in CITY_SHORT.items():
    print(f'  {short!r:15} ← {full!r}')


def filter_to_cities(df: pd.DataFrame, cities: list) -> pd.DataFrame:
    """
    Keep rows where cbsa_title matches any of the given cities.
    Uses partial match on the primary city name (before first comma).
    Adds a 'city_label' column with the short display name.
    """
    primary_names = [c.split(',')[0] for c in cities]
    pattern = '|'.join(primary_names)
    mask = df['cbsa_title'].str.contains(pattern, case=False, na=False)
    filtered = df[mask].copy()

    # Map back to the canonical short name
    def _short(title: str) -> str:
        for c in cities:
            if c.split(',')[0].lower() in title.lower():
                return CITY_SHORT[c]
        return title.split(',')[0]

    filtered['city_label'] = filtered['cbsa_title'].apply(_short)
    print(f'Filtered rows: {len(filtered):,}')
    print(filtered.groupby(['city_label', 'year']).size().reset_index(name='n_rows'))
    return filtered


city_income = filter_to_cities(dkl_income, CITIES)
city_income.head(3)


## 3. Aggregate dkl_bin by Income Bracket

`dkl_bin` is defined per *(variable_group, cbsa_fips, variable)* row in the
DKL output.  Each `variable` corresponds to one income bracket.  We aggregate
the median `dkl_bin` across all block groups within each (city × year × bracket)
combination so the bar height represents a typical block group's bin-level
divergence.


In [ ]:
# Determine which column carries the bracket label
LABEL_COL = None
for candidate in ('label', 'group_label', 'variable_label', 'variable'):
    if candidate in city_income.columns:
        LABEL_COL = candidate
        break
print(f'Using label column: {LABEL_COL!r}')


def aggregate_dkl_bin(df: pd.DataFrame, label_col: str) -> pd.DataFrame:
    """
    Aggregate dkl_bin per (city_label, year, income_bracket).
    Returns a long-format DataFrame ready for dodged bar charts.
    """
    agg = (
        df.groupby(['city_label', 'year', label_col], observed=True)
        ['dkl_bin']
        .agg(median_dkl_bin='median', mean_dkl_bin='mean', count='count')
        .reset_index()
        .rename(columns={label_col: 'bracket'})
    )
    # Preserve original bracket order as much as possible by using the
    # Census variable code if available
    if 'variable' in df.columns:
        var_order = (
            df[['variable', label_col]]
            .drop_duplicates()
            .sort_values('variable')
            [label_col]
            .unique()
            .tolist()
        )
        agg['bracket'] = pd.Categorical(agg['bracket'], categories=var_order, ordered=True)
        agg = agg.sort_values(['city_label', 'year', 'bracket'])

    print(f'Aggregated to {len(agg):,} rows')
    return agg


agg_income = aggregate_dkl_bin(city_income, LABEL_COL)
agg_income.head(10)


## 4. Dodged Bar Charts — dkl_bin by Income Bracket

One figure per city; bars are grouped (dodged) by year so 2010, 2015, and 2020
can be compared side-by-side for every income bracket.


In [ ]:
YEAR_PALETTE = {2010: '#4878CF', 2015: '#6ACC65', 2020: '#D65F5F'}


def plot_dodged_bar_city(agg: pd.DataFrame, city: str,
                          metric: str = 'median_dkl_bin') -> None:
    """
    Produce a dodged bar chart for one city, showing dkl_bin by income bracket
    with one bar cluster per bracket and one bar per year.
    """
    sub = agg[agg['city_label'] == city].copy()
    if sub.empty:
        print(f'No data for city: {city!r}')
        return

    years = sorted(sub['year'].unique())
    brackets = sub['bracket'].cat.categories.tolist() if hasattr(sub['bracket'], 'cat') \
               else sub['bracket'].unique().tolist()

    n_brackets = len(brackets)
    n_years    = len(years)
    bar_width  = 0.75 / n_years
    x = np.arange(n_brackets)

    fig, ax = plt.subplots(figsize=(max(14, n_brackets * 0.7), 6))

    for i, yr in enumerate(years):
        yr_data = sub[sub['year'] == yr].set_index('bracket')[metric]
        vals = [yr_data.get(b, np.nan) for b in brackets]
        offset = (i - (n_years - 1) / 2) * bar_width
        bars = ax.bar(
            x + offset, vals,
            width=bar_width * 0.9,
            color=YEAR_PALETTE.get(yr, f'C{i}'),
            label=str(yr),
            edgecolor='white', linewidth=0.3,
            zorder=2
        )

    ax.set_xticks(x)
    ax.set_xticklabels(
        [str(b) for b in brackets],
        rotation=45, ha='right', fontsize=8
    )
    ax.set_xlabel('Income Bracket', fontsize=10)
    ax.set_ylabel('Median dkl_bin (bits)', fontsize=10)
    ax.set_title(
        f'{city} — DKL Bin by Income Bracket\n'
        f'(median dkl_bin per block group across the CBSA)',
        fontsize=12, fontweight='bold'
    )
    ax.legend(title='Year', fontsize=9)
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.4f'))
    ax.grid(axis='y', linewidth=0.5, alpha=0.7)
    ax.set_axisbelow(True)
    plt.tight_layout()
    plt.show()


for city in sorted(agg_income['city_label'].unique()):
    plot_dodged_bar_city(agg_income, city)


## 5. Four-City Comparison — All Cities in One Figure

Show all four cities on a shared set of subplots for a side-by-side comparison.
Each row is one city; each cluster of bars is one income bracket.


In [ ]:
def plot_dodged_bar_all_cities(agg: pd.DataFrame, metric: str = 'median_dkl_bin') -> None:
    """
    Subplot grid: one row per city, shared bracket axis.
    """
    cities = sorted(agg['city_label'].unique())
    years  = sorted(agg['year'].unique())

    # Determine a unified bracket order
    if hasattr(agg['bracket'], 'cat'):
        brackets = agg['bracket'].cat.categories.tolist()
    else:
        brackets = agg['bracket'].unique().tolist()

    n_brackets = len(brackets)
    n_years    = len(years)
    n_cities   = len(cities)
    bar_width  = 0.75 / n_years
    x = np.arange(n_brackets)

    fig, axes = plt.subplots(
        n_cities, 1,
        figsize=(max(16, n_brackets * 0.75), 5 * n_cities),
        sharex=True
    )
    if n_cities == 1:
        axes = [axes]

    fig.suptitle(
        'DKL Bin by Income Bracket — Four Study Cities',
        fontsize=14, fontweight='bold', y=1.01
    )

    for ax, city in zip(axes, cities):
        sub = agg[agg['city_label'] == city]
        for i, yr in enumerate(years):
            yr_data = sub[sub['year'] == yr].set_index('bracket')[metric]
            vals = [yr_data.get(b, np.nan) for b in brackets]
            offset = (i - (n_years - 1) / 2) * bar_width
            ax.bar(
                x + offset, vals,
                width=bar_width * 0.9,
                color=YEAR_PALETTE.get(yr, f'C{i}'),
                label=str(yr),
                edgecolor='white', linewidth=0.3, zorder=2
            )
        ax.set_ylabel('Median dkl_bin (bits)', fontsize=9)
        ax.set_title(city, fontsize=10, fontweight='bold', loc='left')
        ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.4f'))
        ax.grid(axis='y', linewidth=0.5, alpha=0.7)
        ax.set_axisbelow(True)
        ax.legend(title='Year', fontsize=8, loc='upper right')

    axes[-1].set_xticks(x)
    axes[-1].set_xticklabels(
        [str(b) for b in brackets],
        rotation=45, ha='right', fontsize=8
    )
    axes[-1].set_xlabel('Income Bracket', fontsize=10)

    plt.tight_layout()
    plt.show()


plot_dodged_bar_all_cities(agg_income)


## 6. Normalised dkl_bin — Bracket Share of Total CBSA Divergence

To see which brackets *dominate* income segregation, normalise each bracket's
median `dkl_bin` by the total `dkl_bin` sum within that (city × year).
This gives a percentage-share stacked view.


In [ ]:
def plot_normalised_stacked(agg: pd.DataFrame, metric: str = 'median_dkl_bin') -> None:
    """Stacked 100% bar chart showing each bracket's share of total dkl_bin."""
    cities = sorted(agg['city_label'].unique())
    years  = sorted(agg['year'].unique())

    if hasattr(agg['bracket'], 'cat'):
        brackets = agg['bracket'].cat.categories.tolist()
    else:
        brackets = agg['bracket'].unique().tolist()

    # Pivot: rows = (city, year), cols = bracket
    pivot = agg.pivot_table(
        index=['city_label', 'year'],
        columns='bracket',
        values=metric,
        aggfunc='mean'
    ).reindex(columns=brackets)

    pivot_norm = pivot.div(pivot.sum(axis=1), axis=0) * 100
    pivot_norm.index = [
        f'{city}\n{yr}' for city, yr in pivot_norm.index
    ]

    cmap = plt.get_cmap('tab20', len(brackets))
    colors = [cmap(i) for i in range(len(brackets))]

    fig, ax = plt.subplots(figsize=(max(12, len(pivot_norm) * 1.0), 7))
    bottom = np.zeros(len(pivot_norm))

    for j, bracket in enumerate(brackets):
        if bracket not in pivot_norm.columns:
            continue
        vals = pivot_norm[bracket].fillna(0).values
        ax.bar(range(len(pivot_norm)), vals, bottom=bottom,
               color=colors[j], label=str(bracket),
               edgecolor='white', linewidth=0.2)
        bottom += vals

    ax.set_xticks(range(len(pivot_norm)))
    ax.set_xticklabels(pivot_norm.index, fontsize=8, rotation=30, ha='right')
    ax.set_ylabel('% Share of Total dkl_bin', fontsize=10)
    ax.set_title(
        'Income Bracket Share of Total DKL Bin\n(normalised to 100% per city × year)',
        fontsize=12, fontweight='bold'
    )
    ax.legend(
        bbox_to_anchor=(1.01, 1), loc='upper left',
        fontsize=7, title='Income Bracket'
    )
    ax.set_ylim(0, 100)
    ax.yaxis.set_major_formatter(mticker.PercentFormatter())
    plt.tight_layout()
    plt.show()


plot_normalised_stacked(agg_income)


In [ ]:
# Summary table
summary = (
    agg_income
    .groupby(['city_label', 'year'])['median_dkl_bin']
    .agg(total_dkl=('sum'), max_bracket=('max'), min_bracket=('min'))
    .round(6)
)
summary
